In [1]:
# Check core SDK version number for debugging purposes
import azureml.core
from azureml.core.model import Model
from azureml.core.image import Image
from azureml.core.webservice import Webservice
print("SDK Version:", azureml.core.VERSION)

SDK Version: 0.1.0.1091601


In [2]:
subscription_id = "786d6510-8f1e-4ae7-b55b-5178716e6ac8"
resource_group = "test-anivan"
workspace_name = "workspace-test1"
workspace_region = 'eastus2' # or eastus2euap

In [4]:
# import the Workspace class and check the azureml SDK version# impor 
from azureml.core.workspace import Workspace, WorkspaceException

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region)
ws.get_details()

{'applicationInsights': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.insights/components/workspacinsightslksuficz',
 'containerRegistry': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.containerregistry/registries/workspacacrezbodeet',
 'description': '',
 'friendlyName': 'workspace-test1',
 'id': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourceGroups/test-anivan/providers/Microsoft.MachineLearningServices/workspaces/workspace-test1',
 'identityPrincipalId': '2059178a-03cf-4cc9-ad26-a5d8ba8ee61b',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.keyvault/vaults/workspackeyvaultqkjjihff',
 'location': 'eastus2',
 'name': 'workspace-test1',
 'storageAccount': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/

In [6]:
model = Model.register(model_path = "model.pkl",
                          model_name = "model.pkl",
                          tags = ["pdm"],
                          description = "Predictive Maintenance multi-class classifier",
                          workspace = ws)

Registering model model.pkl


In [10]:
%%writefile score.py

import json
from sklearn.externals import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('model.pkl')
    model = joblib.load(model_path)

def run(rawdata):
    try:
        data = json.loads(rawdata)
        df = pd.DataFrame(data=data)
        result = model.predict(df).tolist()
    except Exception as e:
        result = str(e)
    return result

Overwriting score.py


In [11]:
# this forces IPython to load the latest version
# of the score module

import importlib
# https://docs.python.org/3/library/importlib.html#importlib.invalidate_caches
importlib.invalidate_caches()
import score
importlib.reload(score)

<module 'score' from '/home/laks/notebooks/score.py'>

In [12]:
score.init()

In [13]:
with open('sample.json', 'r') as sample_file:
  sample_json = sample_file.read()

score.run(sample_json)

['', '', 'F1', '', '']

In [14]:
%%writefile env.yml

name: pdm
channels:
  - defaults
dependencies:
  - pip:
    - numpy
    - scikit-learn
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core

Writing env.yml


In [15]:
image = Image.create(name = "pdm-multi-class-image",
                        models = [model],
                        runtime = "python",
                        execution_script = "score.py",
                        conda_file = "env.yml",
                        tags = ["pdm"],
                        description = "Image for Predicitve Maintenance multi-class classifier",
                        workspace = ws)

Creating image


In [16]:
image.wait_for_creation(show_output = True)

Running..................
SucceededImage creation operation finished for image pdm-multi-class-image:11, operation "Succeeded"


In [17]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = ['pdm'], 
                                               description = 'Predicting equipment failure')

In [20]:
service = Webservice.deploy_from_image(name  = 'pdm16',
                               deployment_config = aciconfig,
                               image = image,
                                workspace = ws)
service.wait_for_deployment(show_output = True)

Creating service
Running....................................................................
SucceededACI service creation operation finished, operation "Succeeded"


In [21]:
predictions = service.run(input_data = sample_json)
print(predictions)

['', '', 'F1', '', '']


In [22]:
vars(service)

{'_auth': <azureml.core.authentication.InteractiveLoginAuthentication at 0x7fb786320c50>,
 '_mms_endpoint': 'https://eastus2.modelmanagement.azureml.net/api/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourceGroups/test-anivan/providers/Microsoft.MachineLearningServices/workspaces/workspace-test1/services/pdm16',
 '_operation_endpoint': 'https://eastus2.modelmanagement.azureml.net/api/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourceGroups/test-anivan/providers/Microsoft.MachineLearningServices/workspaces/workspace-test1/operations/32a37b7f-81f9-498e-ab8b-bef647666c97',
 'compute_type': 'ACI',
 'container_resource_requirements': <azureml.core.webservice.aci.ContainerResourceRequirements at 0x7fb7327f2048>,
 'created_time': datetime.datetime(2018, 6, 27, 4, 51, 19, 548660, tzinfo=tzlocal()),
 'description': 'Predicting equipment failure',
 'error': None,
 'image_id': 'pdm-multi-class-image:11',
 'name': 'pdm16',
 'scoring_uri': 'http://23.100.25.248:5001/score',
 'stat